In [31]:
import joblib
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df_model = pd.read_csv("C:/data/Video Game Sales/video_game_sales_com_target.csv")

# Carregar o pipeline salvo
pipeline_rf = joblib.load('C:/ML/modelos/vgchartz_model_random_forest.joblib')

In [32]:
# Separar features e target
X = df_model.drop(columns=['Sucesso'])
y = df_model['Sucesso']

# Dividir os dados (sem aplicar transformações)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [33]:
# Previsões com o pipeline, que já tem pré-processamento embutido
y_pred = pipeline_rf.predict(X_test)

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4751
           1       1.00      0.99      0.99       439

    accuracy                           1.00      5190
   macro avg       1.00      0.99      1.00      5190
weighted avg       1.00      1.00      1.00      5190



In [35]:
df_completo = df_model.loc[X_test.index].copy()
df_completo['Previsao_Sucesso'] = y_pred

## **Análises**

### **Quais gêneros têm maior chance de sucesso?**

Avaliei a **proporção de jogos bem-sucedidos por gênero**, com base nas previsões geradas pelo modelo de Machine Learning. Para isso, calculei a média da variável `Previsao_Sucesso` agrupada por gênero.

Os resultados mostram que:

- O gênero **Sandbox** se destacou com **100% de chance de sucesso**, sendo o único com todas as amostras classificadas como bem-sucedidas.
- Em seguida, vieram os gêneros **Action-Adventure (21%)**, **Shooter (18%)** e **MMO (12%)**, apresentando desempenho consideravelmente acima da média geral.
- Gêneros como **Party**, **Education**, **Board Game** e **Visual Novel** apresentaram **0% de chance de sucesso**, ou seja, nenhuma amostra desses gêneros foi prevista como bem-sucedida.

Esse resultado sugere que certos gêneros têm maior potencial de sucesso comercial, o que pode refletir tendências de mercado, preferências do público ou estratégias adotadas pelos desenvolvedores.

> 🔎 **Observação**: O modelo pode estar influenciado pelo volume e equilíbrio de amostras em cada gênero, bem como por possíveis correlações com variáveis como plataforma, data de lançamento e região de vendas. Por isso, os dados devem ser interpretados com cautela.


In [36]:
df_genero_sucesso = df_completo.groupby('genre')['Previsao_Sucesso'].mean().sort_values(ascending=False).round(2)
print(df_genero_sucesso)

genre
Sandbox             1.00
Action-Adventure    0.21
Shooter             0.18
MMO                 0.12
Sports              0.11
Fighting            0.10
Racing              0.09
Action              0.09
Misc                0.07
Simulation          0.07
Platform            0.07
Role-Playing        0.05
Adventure           0.04
Music               0.02
Puzzle              0.02
Strategy            0.02
Party               0.00
Education           0.00
Board Game          0.00
Visual Novel        0.00
Name: Previsao_Sucesso, dtype: float64


### **Análise de acurácia por Gênero**

Avaliei a acurácia por gênero comparando as previsões do modelo com os rótulos reais dos dados de teste.

In [37]:
df_avaliacao = df_completo.copy()
df_avaliacao['Real_Sucesso'] = y_test

# Calculando acurácia por plataforma
df_acerto_por_genero = df_avaliacao.groupby('genre').apply(
    lambda df: (df['Previsao_Sucesso'] == df['Real_Sucesso']).mean().round(2)
).sort_values(ascending=False)

print(df_acerto_por_genero)

genre
Action              1.00
Action-Adventure    1.00
Strategy            1.00
Sports              1.00
Simulation          1.00
Shooter             1.00
Sandbox             1.00
Role-Playing        1.00
Racing              1.00
Platform            1.00
Party               1.00
Music               1.00
Misc                1.00
MMO                 1.00
Fighting            1.00
Education           1.00
Board Game          1.00
Adventure           1.00
Visual Novel        1.00
Puzzle              0.99
dtype: float64


C:\Users\fabio\AppData\Local\Temp\ipykernel_22452\2957270668.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_acerto_por_genero = df_avaliacao.groupby('genre').apply(


### **Análise da Chance de Sucesso e Acurácia por Gênero**

In [38]:
# Unido os dois Series em um único DataFrame
df_analise_genero = pd.concat([df_genero_sucesso, df_acerto_por_genero], axis=1)

# Renomeando colunas
df_analise_genero.columns = ['Chance de Sucesso', 'Acurácia']

# Reset do índice para deixar o gênero como coluna comum
df_analise_genero = df_analise_genero.reset_index()
df_analise_genero = df_analise_genero.rename(columns={'index': 'Gênero'})

# Ordenar por Chance de Sucesso, do maior para o menor
df_analise_genero = df_analise_genero.sort_values(by='Chance de Sucesso', ascending=False)

print(df_analise_genero)

               genre  Chance de Sucesso  Acurácia
0            Sandbox               1.00      1.00
1   Action-Adventure               0.21      1.00
2            Shooter               0.18      1.00
3                MMO               0.12      1.00
4             Sports               0.11      1.00
5           Fighting               0.10      1.00
6             Racing               0.09      1.00
7             Action               0.09      1.00
9         Simulation               0.07      1.00
10          Platform               0.07      1.00
8               Misc               0.07      1.00
11      Role-Playing               0.05      1.00
12         Adventure               0.04      1.00
13             Music               0.02      1.00
14            Puzzle               0.02      0.99
15          Strategy               0.02      1.00
16             Party               0.00      1.00
17         Education               0.00      1.00
18        Board Game               0.00      1.00


### **Quais consoles apresentaram maior proporção de jogos bem-sucedidos?**

Avaliei a proporção de sucesso prevista para os jogos em cada console, com base nas previsões do modelo. Os consoles que apresentaram maior proporção de jogos bem-sucedidos foram o **NES** (47%) e o **Mega Drive (GEN)** com 43%. Já consoles mais recentes como **Xbox 360**, **PlayStation 4** e **PlayStation 3** tiveram proporções mais modestas, entre 13% e 18%. A maioria dos demais consoles obteve proporções de sucesso inferiores a 10%.

In [39]:
df_console_sucesso = df_completo.groupby('console')['Previsao_Sucesso'].mean().sort_values(ascending=False).round(2)
print(df_console_sucesso)

console
NES     0.47
GEN     0.43
X360    0.18
PS4     0.16
GB      0.15
PS3     0.15
PS      0.14
XOne    0.13
PS2     0.10
SNES    0.09
Wii     0.07
N64     0.06
DC      0.06
GBA     0.05
XB      0.05
PC      0.05
PSP     0.04
DS      0.03
GC      0.03
NS      0.03
SAT     0.02
PSV     0.02
3DS     0.01
WiiU    0.00
GG      0.00
VC      0.00
SCD     0.00
NG      0.00
3DO     0.00
Name: Previsao_Sucesso, dtype: float64


### **Análise de acurácia por Console**

Avaliei a acurácia das previsões do modelo para cada console, verificando a proporção de acertos entre as previsões e os valores reais de sucesso. No resultado, a maioria dos consoles obteve acurácia de **100%**, indicando que o modelo acertou todas as previsões para esses grupos. Apenas três consoles — **N64**,

In [40]:
df_avaliacao = df_completo.copy()
df_avaliacao['Real_Sucesso'] = y_test

# Calculando acurácia por console
df_acerto_por_console = df_avaliacao.groupby('console').apply(
    lambda df: (df['Previsao_Sucesso'] == df['Real_Sucesso']).mean().round(2)
).sort_values(ascending=False)

print(df_acerto_por_console)

console
3DO     1.00
PS2     1.00
XB      1.00
X360    1.00
WiiU    1.00
Wii     1.00
VC      1.00
SNES    1.00
SCD     1.00
SAT     1.00
PSV     1.00
PSP     1.00
PS4     1.00
PS3     1.00
XOne    1.00
3DS     1.00
PC      1.00
NS      1.00
NG      1.00
NES     1.00
GG      1.00
GEN     1.00
GBA     1.00
GB      1.00
DS      1.00
DC      1.00
N64     0.99
GC      0.99
PS      0.99
dtype: float64


C:\Users\fabio\AppData\Local\Temp\ipykernel_22452\3650702498.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_acerto_por_console = df_avaliacao.groupby('console').apply(


### **Análise da Chance de Sucesso e Acurácia por Console**

In [42]:
# Unido os dois Series em um único DataFrame
df_analise_console = pd.concat([df_console_sucesso, df_acerto_por_console], axis=1)

# Renomeando colunas
df_analise_console.columns = ['Chance de Sucesso', 'Acurácia']

# Reset do índice para deixar o gênero como coluna comum
df_analise_console = df_analise_console.reset_index()
df_analise_console = df_analise_console.rename(columns={'index': 'Gênero'})

# Ordenar por Chance de Sucesso, do maior para o menor
df_analise_console = df_analise_console.sort_values(by='Chance de Sucesso', ascending=False)

print(df_analise_console)

   console  Chance de Sucesso  Acurácia
0      NES               0.47      1.00
1      GEN               0.43      1.00
2     X360               0.18      1.00
3      PS4               0.16      1.00
4       GB               0.15      1.00
5      PS3               0.15      1.00
6       PS               0.14      0.99
7     XOne               0.13      1.00
8      PS2               0.10      1.00
9     SNES               0.09      1.00
10     Wii               0.07      1.00
11     N64               0.06      0.99
12      DC               0.06      1.00
15      PC               0.05      1.00
14      XB               0.05      1.00
13     GBA               0.05      1.00
16     PSP               0.04      1.00
17      DS               0.03      1.00
18      GC               0.03      0.99
19      NS               0.03      1.00
20     SAT               0.02      1.00
21     PSV               0.02      1.00
22     3DS               0.01      1.00
23    WiiU               0.00      1.00


### **Quais fabricantes apresentaram maior proporção de jogos bem-sucedidos?**
Calculei a média das previsões de sucesso para os jogos de cada fabricante, revelando quais apresentaram maior proporção de títulos bem-sucedidos segundo o modelo. A **Microsoft** lidera com 13% dos jogos previstos como sucesso, seguida por **Sony** com 10%. **Sega**, **Nintendo** e o grupo classificado como **Outros** aparecem com proporções menores, enquanto a **SNK** não teve jogos classificados como bem-sucedidos.

In [29]:
df_fabricante_sucesso = df_completo.groupby('console_fabricator')['Previsao_Sucesso'].mean().sort_values(ascending=False).round(2)
print(df_fabricante_sucesso)

console_fabricator
Microsoft    0.13
Sony         0.10
Sega         0.07
Nintendo     0.05
Outros       0.05
SNK          0.00
Name: Previsao_Sucesso, dtype: float64


### **Análise de acurácia por Fabricante**
Avaliei a acurácia das previsões do modelo agrupando os resultados por fabricante de console. Todos os fabricantes apresentaram **100% de acerto** nas amostras testadas, o que indica que o modelo conseguiu prever corretamente o sucesso ou fracasso dos jogos de forma consistente em todos os grupos.


In [30]:
df_avaliacao = df_completo.copy()
df_avaliacao['Real_Sucesso'] = y_test

# Calculando acurácia por fabricante de console
df_acerto_por_fabricante_console = df_avaliacao.groupby('console_fabricator').apply(
    lambda df: (df['Previsao_Sucesso'] == df['Real_Sucesso']).mean().round(2)
).sort_values(ascending=False)

print(df_acerto_por_fabricante_console)

console_fabricator
Microsoft    1.0
Nintendo     1.0
Outros       1.0
SNK          1.0
Sega         1.0
Sony         1.0
dtype: float64


C:\Users\fabio\AppData\Local\Temp\ipykernel_22452\123579835.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acerto_por_fabricante_console = df_avaliacao.groupby('console_fabricator').apply(


### **Análise da Chance de Sucesso e Acurácia por Fabricante de Console**

In [44]:
# Unido os dois Series em um único DataFrame
df_analise_fabricante_console = pd.concat([df_console_sucesso, df_acerto_por_console], axis=1)

# Renomeando colunas
df_analise_fabricante_console.columns = ['Chance de Sucesso', 'Acurácia']

# Reset do índice para deixar o gênero como coluna comum
df_analise_fabricante_console = df_analise_fabricante_console.reset_index()
df_analise_fabricante_console = df_analise_fabricante_console.rename(columns={'index': 'Gênero'})

# Ordenar por Chance de Sucesso, do maior para o menor
df_analise_fabricante_console = df_analise_fabricante_console.sort_values(by='Chance de Sucesso', ascending=False)

print(df_analise_fabricante_console)

   console  Chance de Sucesso  Acurácia
0      NES               0.47      1.00
1      GEN               0.43      1.00
2     X360               0.18      1.00
3      PS4               0.16      1.00
4       GB               0.15      1.00
5      PS3               0.15      1.00
6       PS               0.14      0.99
7     XOne               0.13      1.00
8      PS2               0.10      1.00
9     SNES               0.09      1.00
10     Wii               0.07      1.00
11     N64               0.06      0.99
12      DC               0.06      1.00
15      PC               0.05      1.00
14      XB               0.05      1.00
13     GBA               0.05      1.00
16     PSP               0.04      1.00
17      DS               0.03      1.00
18      GC               0.03      0.99
19      NS               0.03      1.00
20     SAT               0.02      1.00
21     PSV               0.02      1.00
22     3DS               0.01      1.00
23    WiiU               0.00      1.00


### **Qual região apresenta maior proporção de jogos bem-sucedidos?**
Os jogos classificados como sucesso pelo modelo apresentam maior desempenho comercial na América do Norte e Europa, enquanto as vendas no Japão e em outras regiões são significativamente menores.


In [49]:
df_analise_regiao_sucesso = df_completo[df_completo['Previsao_Sucesso'] == 1]

df_analise_regiao_sucesso = df_analise_regiao_sucesso[['na_sales', 'pal_sales', 'jp_sales', 'other_sales']].mean().round(2)
print("Média de vendas em milhõs: ")
print(df_analise_regiao_sucesso)

Média de vendas em milhõs: 
na_sales       1.10
pal_sales      0.86
jp_sales       0.10
other_sales    0.28
dtype: float64


Apesar do Japão ser uma potência em desenvolvimento e consumo de jogos, os títulos bem-sucedidos segundo nosso modelo apresentaram, em média, vendas significativamente menores nessa região. Isso pode ser explicado pelo foco local de muitos desenvolvedores japoneses, que criam jogos voltados majoritariamente para o público interno, sem priorizar a tradução ou a distribuição internacional. Com isso, esses jogos acabam tendo menor expressão nos dados globais e, por consequência, menor impacto na métrica de sucesso usada pelo modelo.

### **Conclusão geral das análises de acurácia**

As análises de acurácia segmentadas por gênero, console e fabricante revelam que o modelo de classificação teve um desempenho **altamente satisfatório** nas previsões. Em praticamente todas as segmentações, as taxas de acerto ficaram próximas ou iguais a 100%. Isso pode indicar tanto um **modelo bem treinado**, quanto um possível **desequilíbrio no dataset** (poucos exemplos de insucesso, por exemplo) — um fator que deve ser melhor investigado com a matriz de confusão e outras métricas como F1-score.

As segmentações por `publisher` e `developer`, embora possíveis, foram evitadas nesta etapa por conterem muitos valores únicos, o que **compromete a clareza visual e interpretativa** dos resultados.

As informações obtidas até aqui serão exportadas para o Power BI a fim de construir visualizações que auxiliem na identificação de padrões relacionados ao sucesso dos jogos.


### **Exportação de dados**

In [51]:
df_analise_genero.to_csv('analise_genero_sucesso_acuracia.csv', index=True, header=0)
df_analise_console.to_csv('analise_console_sucesso_acuracia.csv', index=True, header=0)
df_analise_fabricante_console.to_csv('analise_fabricante_previsao_acuracia.csv', index=True, header=0)
df_analise_regiao_sucesso.to_csv('analise_regiao_sucesso.csv', index=True, header=0)